<div class="page-header"><h1 class="alert alert-info">Data Camp: Stock Prediction<br/>
<small>Gustavo Castro, Lucas Furquim, Francisco Ribeiro, Alvaro Serra<br/>

<h1 class="alert alert-success">Introduction</h1>

Managing and creating good <a href = "http://www.investopedia.com/terms/p/portfoliomanagement.asp"> portfolios </a> are main works in the financial world, specially at the Quantitative Asset Management sector. In this context, one needs to decide which portfolio allocation that will give the best future return.

To do so, it is vital to be able to forecast some stock behaviors and their variances. For more detailed information and explanation please refer to: <a href = "http://pubsonline.informs.org/doi/abs/10.1287/mnsc.2013.1838" > Risk Premium 
forecast </a> and <a href ="http://cims.nyu.edu/~almgren/timeseries/Vol_Forecast1.pdf"> GARCH Model </a>.

The goal of this challenge is to predict the <a href = "http://www.investopedia.com/terms/s/sp500.asp"> SP500 index </a>  behavior using some market data and to mine the different interactions this index might have with the proposed features.

<h1 class="alert alert-success"> Prediction task and Hints</h1>

As explained, the goal is to be able to predict the behavior of the index during the year of 2015. To do so, a considerable amount of historical data is offered. 

It is important to be aware of the existence of NaN values in the database, specially in older periods (before 1900). We thus strongly suggest that the student initially ignores the data older than january 1950 to avoid initial annoying problems with database empty and Nan cells. 

We also emphasize that the student is completly free to define his predicton model and the time interval that will be considered to calibrate it. 

As always in the machine learning context, we accentuate the importance of a proper feature analysis, their relevances, signifcations and impacts under this prediction context. To stimulate this work, we propose, at the Data section of this notebook, the use of some new features and we strongly advice the wise choice of the features' relevances and the creation of others that the student may find relevant.



<h1 class="alert alert-success"> Imports and Initial Setup </h1>

## Tools & Setup

- *The simple way*: Install the Anaconda python distribution https://www.continuum.io/downloads
- *The fine-grained way:* Install each of the following tools
    - Python
    - Jupyter
    - Scikit-learn
    - Pandas

## Imports


In [2]:
import numpy  as np
import pandas as pd

<h1 class="alert alert-success"> Data </h1>
To do our study of the SP500 index (graphic bellow), we propose, in this project, a dataset of monthly market indices since january 1871 until december 2015. A detailed explanation is given below:

<img src="Image/SP500.jpg">

<h1 class="alert alert-success">Data description</h1>

The following table contains the description of the different columns in the dataset

In [3]:
meta_brute = pd.read_csv('Data/BruteMetaData.csv')
meta_brute

,Unnamed: 0,Feature_name,Feature_definition
0,0,yyyymm,year and month of the concerned part of the data.
1,1,MD,monthly dividend paid.
2,2,D12,12-month moving sum of dividends.
3,3,E12,12-month moving sum of earnings.
4,4,B/M,ratio of the accounting book value to the mark...
5,5,TBL,interest rate on a three-month Treasury bill (...
6,6,AAA,AAA-rated corporate bond yield.
7,7,BAA,BAA-rated corporate bond yield.
8,8,LTY,long-term government bond yield.
9,9,NTIS,ratio of the 12-month moving sum of net equity...


In [17]:
train_brute = pd.read_csv('Data/BruteTrainData.csv')
train_brute.head(10)

,yyyymm,MD,D12,E12,B/M,TBL,AAA,BAA,LTY,NTIS,RFREE,INFL,LTR,CORPR,SVAR
0,187101,0.0217,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004955,NaN,NaN,NaN,NaN
1,187102,0.0217,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004514,NaN,NaN,NaN,NaN
2,187103,0.0217,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004243,NaN,NaN,NaN,NaN
3,187104,0.0217,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004632,NaN,NaN,NaN,NaN
4,187105,0.0217,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003691,NaN,NaN,NaN,NaN
5,187106,0.0217,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003368,NaN,NaN,NaN,NaN
6,187107,0.0217,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003250,NaN,NaN,NaN,NaN
7,187108,0.0217,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003706,NaN,NaN,NaN,NaN
8,187109,0.0217,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004544,NaN,NaN,NaN,NaN
9,187110,0.0217,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.007016,NaN,NaN,NaN,NaN


In [18]:
train_brute.tail(10)

,yyyymm,MD,D12,E12,B/M,TBL,AAA,BAA,LTY,NTIS,RFREE,INFL,LTR,CORPR,SVAR
1718,201403,3.0968,36.2300,100.8500,0.350616,0.0005,0.0438,0.0506,0.0331,0.016464,0.000042,0.001171,0.0063,0.0062,0.000869
1719,201404,3.2488,36.6133,101.6067,0.348011,0.0003,0.0424,0.0490,0.0324,0.016697,0.000025,0.002535,0.0181,0.0160,0.001363
1720,201405,3.2489,36.9967,102.3633,0.345173,0.0003,0.0416,0.0476,0.0307,0.014027,0.000025,0.001302,0.0279,0.0188,0.000555
1721,201406,3.2488,37.3800,103.1200,0.342928,0.0004,0.0425,0.0480,0.0307,0.013931,0.000033,0.000899,-0.0025,0.0020,0.000282
1722,201407,3.3168,37.7500,104.0667,0.348380,0.0003,0.0416,0.0473,0.0301,0.008473,0.000025,0.001025,0.0057,0.0024,0.000949
1723,201408,3.3169,38.1200,105.0133,0.337476,0.0003,0.0408,0.0469,0.0281,0.006770,0.000025,-0.000405,0.0369,0.0356,0.000546
1724,201409,3.3168,38.4900,105.9600,0.338576,0.0002,0.0411,0.0480,0.0295,0.008025,0.000017,0.001463,-0.0170,-0.0271,0.000721
1725,201410,3.4841,38.8067,104.7433,0.331808,0.0002,0.0392,0.0469,0.0273,0.008069,0.000017,0.000594,0.0300,0.0225,0.002944
1726,201411,3.4842,39.1233,103.5267,0.323662,0.0002,0.0392,0.0479,0.0263,0.006926,0.000017,-0.001641,0.0286,0.0173,0.000150
1727,201412,3.4841,39.4400,102.3100,0.323756,0.0003,0.0379,0.0474,0.0240,0.005642,0.000025,-0.003359,0.0290,0.0183,0.001993


In [19]:
train_brute.describe()

,yyyymm,MD,D12,E12,B/M,TBL,AAA,BAA,LTY,NTIS,RFREE,INFL,LTR,CORPR,SVAR
count,1728.000000,1728.000000,1728.000000,1728.000000,1126.000000,1139.000000,1152.000000,1152.000000,1152.000000,1057.000000,1728.000000,1223.000000,1068.000000,1068.000000,1559.000000
mean,194256.500000,0.406372,4.748414,11.086228,0.573928,0.035268,0.058306,0.070213,0.051472,0.018374,0.003094,0.002609,0.004870,0.005154,0.002528
std,4158.026443,0.663870,7.698757,20.832073,0.258185,0.029992,0.026970,0.028536,0.026897,0.025268,0.002145,0.006198,0.024180,0.021541,0.004962
min,187101.000000,0.007500,0.180000,0.160000,0.120510,0.000100,0.024600,0.029400,0.018200,-0.057750,0.000008,-0.032088,-0.112400,-0.094900,0.000000
25%,190678.750000,0.027600,0.400000,0.525850,0.377188,0.009500,0.038100,0.048800,0.030250,0.007058,0.001775,0.000000,-0.006800,-0.003800,0.000728
50%,194256.500000,0.070850,0.781650,1.245000,0.546202,0.032100,0.051100,0.064650,0.043200,0.017783,0.003017,0.002375,0.003300,0.004000,0.001237
75%,197834.250000,0.446700,5.080825,11.152525,0.742913,0.051650,0.074850,0.084800,0.067325,0.027765,0.004059,0.004934,0.015925,0.013400,0.002253
max,201412.000000,3.484200,39.440000,105.960000,2.028478,0.163000,0.154900,0.171800,0.148200,0.177040,0.013583,0.057357,0.152300,0.156000,0.070945


We found out that the following treated database can also be very useful

The following table contains the description of the different columns in the treated dataset

In [21]:
meta_treated = pd.read_csv('Data/TreatedMetaData.csv')
meta_treated

,Unnamed: 0,Feature_name,Feature_definition
0,0,yyyymm,year and month of the concerned part of the data.
1,1,log(DE),log of the 12-month moving sum of dividends di...
2,2,SVAR,monthly sum of squared daily returns.
3,3,B/M,ratio of the accounting book value to the mark...
4,4,NTIS,ratio of the 12-month moving sum of net equity...
5,5,TBL,interest rate on a three-month Treasury bill (...
6,6,LTY,long-term government bond yield.
7,7,LTR,return on long-term government bonds.
8,8,TMS,long-term yield minus the Treasury bill rate (...
9,9,DFY,difference between BAA- and AAA-rated corporat...


In [29]:
train_treated = pd.read_csv('Data/TreatedTrainData.csv')
train_treated.head(10)

,yyyymm,log(DE),SVAR,B/M,NTIS,TBL,LTY,LTR,TMS,DFY,DFR,INFL
0,187101,-0.187087,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,NaN
1,187102,-0.187087,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,NaN
2,187103,-0.187087,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,NaN
3,187104,-0.187087,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,NaN
4,187105,-0.187087,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,NaN
5,187106,-0.187087,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,NaN
6,187107,-0.187087,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,NaN
7,187108,-0.187087,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,NaN
8,187109,-0.187087,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,NaN
9,187110,-0.187087,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,NaN


In [30]:
train_treated.tail(10)

,yyyymm,log(DE),SVAR,B/M,NTIS,TBL,LTY,LTR,TMS,DFY,DFR,INFL
1718,201403,-0.444608,0.000869,0.350616,0.016464,0.0005,0.0331,0.0063,0.0326,-0.0068,-0.0001,0.000786
1719,201404,-0.443283,0.001363,0.348011,0.016697,0.0003,0.0324,0.0181,0.0321,-0.0066,-0.0021,0.001171
1720,201405,-0.441982,0.000555,0.345173,0.014027,0.0003,0.0307,0.0279,0.0304,-0.006,-0.0091,0.002535
1721,201406,-0.440704,0.000282,0.342928,0.013931,0.0004,0.0307,-0.0025,0.0303,-0.0055,0.0045,0.001302
1722,201407,-0.440395,0.000949,0.348380,0.008473,0.0003,0.0301,0.0057,0.0298,-0.0057,-0.0033,0.000899
1723,201408,-0.440092,0.000546,0.337476,0.006770,0.0003,0.0281,0.0369,0.0278,-0.0061,-0.0013,0.001025
1724,201409,-0.439794,0.000721,0.338576,0.008025,0.0002,0.0295,-0.0170,0.0293,-0.0069,-0.0101,-0.000405
1725,201410,-0.431220,0.002944,0.331808,0.008069,0.0002,0.0273,0.0300,0.0271,-0.0077,-0.0075,0.001463
1726,201411,-0.422616,0.000150,0.323662,0.006926,0.0002,0.0263,0.0286,0.0261,-0.0087,-0.0113,0.000594
1727,201412,-0.413981,0.001993,0.323756,0.005642,0.0003,0.0240,0.0290,0.0237,-0.0095,-0.0107,-0.001641


In [31]:
train_treated.describe()

,yyyymm,log(DE),SVAR,B/M,NTIS,TBL,LTY,LTR,INFL
count,1728.000000,1728.000000,1559.000000,1126.000000,1057.000000,1139.000000,1152.000000,1068.000000,1222.000000
mean,194256.500000,-0.234766,0.002528,0.573928,0.018374,0.035268,0.051472,0.004870,0.002614
std,4158.026443,0.137271,0.004962,0.258185,0.025268,0.029992,0.026897,0.024180,0.006199
min,187101.000000,-0.540353,0.000000,0.120510,-0.057750,0.000100,0.018200,-0.112400,-0.032088
25%,190678.750000,-0.308070,0.000728,0.377188,0.007058,0.009500,0.030250,-0.006800,0.000000
50%,194256.500000,-0.238773,0.001237,0.546202,0.017783,0.032100,0.043200,0.003300,0.002375
75%,197834.250000,-0.163363,0.002253,0.742913,0.027765,0.051650,0.067325,0.015925,0.004934
max,201412.000000,0.599090,0.070945,2.028478,0.177040,0.163000,0.148200,0.152300,0.057357


<h1 class="alert alert-success">The prediction model</h1>

We are going to follow the scikit-learn API specs in order to define a `FeatureExtractor` and a `Regressor`.

## The feature extractor

In <code>feature_extractor.py</code> you will define a class called <code>FeatureExtractor</code>. Its main <code>transform</code> method takes a pandas <b>DataFrame</b> and outputs a <b>numpy array</b>.

- The `FeatureExtractor` inherits from `TransformerMixin`.
- It implements a `fit` (optional) and a `transform` method. 

In [7]:
import pandas as pd
from sklearn.base import TransformerMixin

class FeatureExtractor(TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X_df, y):
        return self

    def transform(self, X_df):
        X_df['yyyymm'] -= X_df['yyyymm'][0]
        X_df['yyyymm'] /= X_df['yyyymm'].iget(-1)
        return X_df.values

## The regressor

- The `Regressor` inherits from `BaseEstimator`,
- The `__init__()` function initiates all of the arguments and configurations. 
- The regressor must implement a `fit()` and  a `predict()` function.

In [8]:
from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer
from sklearn.ensemble import ExtraTreesRegressor


class Regressor(BaseEstimator):
    def __init__(self):
        self.reg = make_pipeline(
            Imputer(strategy='median'),
            ExtraTreesRegressor(n_estimators=10))

    def fit(self, X, y):
        return self.reg.fit(X, y)

    def predict(self, X):
        return self.reg.predict(X)


    def predict(self, X):
        return self.reg.predict(X)

## Unit testing

It is <b><span style="color:red">important that you test your submission files before submitting them</span></b>. For this we
provide a unit test. Place the python file <code>regressor.py</code>, the data <code>public_train.csv</code>, and the 
<code>user_test_submission.py</code></a> in a directory and run 

<code>python user_test_submission.py</code>

If it runs and prints 
<code>
rmse =  [some_number]
rmse =  [some_number]
</code>
you can submit the code.

In [6]:
!python user_test_submission.py

Traceback (most recent call last):
  File "user_test_submission.py", line 5, in <module>
    import feature_extractor
  File "C:\Users\Francisco\Documents\X\3A\data camp\project\feature_extractor.py", line 12
    def transform(self, X_df):
                             ^
TabError: inconsistent use of tabs and spaces in indentation
